In [1]:
import csv
from collections import defaultdict
from collections import Counter
import dendropy
from Bio import SeqIO
import tqdm 
import random

In [2]:
serotypes = ["DENV1", "DENV2", "DENV3", "DENV4"]

In [14]:
minor_lineages = defaultdict(list)
major_lineages = defaultdict(list)
genotypes = defaultdict(list)
coverage = {}
in_minors = set()
in_majors = set()
with open("sequence_metadata.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        
        if l['minor_lineage'] != "":
            in_minors.add(l['fasta_name'])
            minor_parts = l['minor_lineage'].split(".")
            if len(minor_parts) > 2:
                previous = minor_parts[0]
                for number in minor_parts[1:]:
                    current = f"{previous}.{number}"
                    minor_lineages[current].append(l['fasta_name'])
                    previous = current
            else:
                minor_lineages[l["minor_lineage"]].append(l['fasta_name'])

                    
        
        if l['major_lineage'] != "":
            major_lineages[l['major_lineage']].append(l['fasta_name'])
            in_majors.add(l['fasta_name'])
        if l['genotype'] != "Unassigned":
            genotypes[l['genotype']].append(l['fasta_name'])
            
        coverage[l['fasta_name']] = l['trimmed_coverage']

In [15]:
total_minor = list(minor_lineages.keys())
total_major = list(major_lineages.keys())
total_genotype = list(genotypes.keys())


In [16]:
tree_dict={}
for sero in ["DENV1", "DENV2", "DENV3", "DENV4"]:
    tree_dict[sero] = dendropy.Tree.get(path=f"{sero}.tree", schema="nexus", preserve_underscores=True)

In [ ]:
for sero in serotypes:
    print(sero)
    tree = dendropy.Tree.get(path=f"{sero}.tree", schema="nexus", preserve_underscores=True)
    print("tree in")
    matrix = dendropy.PhylogeneticDistanceMatrix.from_tree(tree)
    
    matrix.write_csv(f"{sero}_distance_matrix.csv")


In [ ]:
distances_from_root = defaultdict(dict)
for sero in serotypes:
    inner_dict = {}
    print(sero)
    tree = dendropy.Tree.get(path=f"{sero}.tree", schema="nexus", preserve_underscores=True)
    for leaf in tqdm.tqdm(tree.leaf_node_iter()):
        inner_dict[leaf.taxon.label] = leaf.distance_from_root()
        
    distances_from_root[sero] = inner_dict

In [ ]:
def get_largest_dist_pairs(already_in, seq_list, distances):
    
    to_test = []
    for i in seq_list:
        if i not in already_in:
            to_test.append(i)
            
    new_distances = {}
    for i in already_in:
        for j in to_test:
            if ((i,j)) in distances:
                key = ((i,j))
            else:
                key = ((j,i))
                
            if j not in new_distances:
                new_distances[j] = distances[key]
            else:
                new_distances[j] += distances[key]
                
    new_seq = list({k:v for k,v in sorted(new_distances.items(), key = lambda x: x[1], reverse=True)})[0]
    already_in.add(new_seq)
    
    if len(already_in) < 5:
        get_largest_dist_pairs(already_in, seq_list, distances)
        return already_in
    else:
        return already_in

In [18]:
def get_sets(lineage_list, sero):
    
    distances = {}
    done = set()
    with open(f"{sero}_distance_matrix.csv") as f:
        data = csv.DictReader(f)
        headers = data.fieldnames
        for l in data:
            seq = l['sequence']
            if l['sequence'] in lineage_list:
                for seq2 in headers:
                    if seq2 != "sequence" and seq2 in lineage_list:
                        pair = ((seq,seq2))
                        if pair not in done:
                            distances[pair] = float(l[seq2])  
                            done.add(pair)
                            done.add((seq2,seq))  
                            
    first_pair = list({k:v for k,v in sorted(distances.items(), key = lambda x: x[1], reverse=True)})[0]
    already_in = set()
    for i in first_pair:
        already_in.add(i)
    final_set = get_largest_dist_pairs(already_in, lineage_list, distances)
    
    return final_set
        
        

## major and genotypes

In [36]:
major_lin_sets = defaultdict(set)

for lin, seq_list in tqdm.tqdm(major_lineages.items()):
    serotype = f"DENV{lin[0]}"
    if len(seq_list) > 5:
        new_seq_list = set([x for x in seq_list if float(coverage[x]) >= 0.9])
        if len(new_seq_list) > 5:
            major_lin_sets[lin] = get_sets(new_seq_list, serotype)
        else:
            major_lin_sets[lin] = set(new_seq_list)
    else:
        major_lin_sets[lin] = set(seq_list)


    tree = tree_dict[serotype]
    mrca = tree.mrca(taxon_labels=seq_list)
    found_sequence = False
    for i in mrca.child_nodes():
        if i.taxon:
            basal = i.taxon.label
            found_sequence = True
            
    if not found_sequence:
        dist_dict = {x:distances_from_root[serotype][x] for x in major_lineages[lin]}
        basal = min(dist_dict, key=dist_dict.get)

    major_lin_sets[lin].add(basal)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [05:33<00:00,  6.54s/it]


In [37]:
with open("representatives_major_lineage.csv", 'w') as fw:
    
    fw.write('taxon,major_lineage\n')
    
    for k,v in major_lin_sets.items():
        for i in v:
            fw.write(f"{i},{k}\n")

In [38]:
genotype_sets = defaultdict(set)

fw = open("representatives_genotype.csv", 'w')
fw.write('taxon,genotype\n')
fw.close()

for lin, seq_list in tqdm.tqdm(genotypes.items()):
    serotype = f"DENV{lin[0]}"
    if len(seq_list) > 5:
        new_seq_list = [x for x in seq_list if float(coverage[x]) >= 0.9]
        if len(new_seq_list) > 5:
            genotype_sets[lin] = get_sets(new_seq_list, serotype)
        else:
            genotype_sets[lin] = set(new_seq_list)
    else:
        genotype_sets[lin] = set(seq_list)
    
        
    tree = tree_dict[serotype]
    mrca = tree.mrca(taxon_labels=seq_list)
    found_sequence = False
    for i in mrca.child_nodes():
        if i.taxon:
            basal = i.taxon.label
            found_sequence = True
    
    if not found_sequence:
        dist_dict = {x:distances_from_root[serotype][x] for x in genotypes[lin]}
        basal = min(dist_dict, key=dist_dict.get)
    
    genotype_sets[lin].add(basal)


    with open("representatives_genotype.csv", 'a') as fw:
        for i in genotype_sets[lin]:
            fw.write(f"{i},{lin}\n")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [13:37<00:00, 40.85s/it]


In [39]:
## make alignments and annotations files

geno_seq_names = defaultdict(list)
with open("representatives_genotype.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        sero = f"DENV{l['genotype'][0]}"
        geno_seq_names[sero].append(l['taxon'])

major_seq_names = defaultdict(list)
with open("representatives_major_lineage.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        sero = f"DENV{l['major_lineage'][0]}"
        major_seq_names[sero].append(l['taxon'])

        
for sero, names in major_seq_names.items():
    new_aln = open(f"{sero}_major_lineage.fasta",'w')
    for seq in SeqIO.parse(f"{sero}_final.fasta", "fasta"):
        if seq.id in names:
            SeqIO.write(seq, new_aln, "fasta")
            
    new_aln.close()
    
for sero, names in geno_seq_names.items():
    new_aln = open(f"{sero}_genotype.fasta",'w')
    for seq in SeqIO.parse(f"{sero}_final.fasta", "fasta"):
        if seq.id in names:
            SeqIO.write(seq, new_aln, "fasta")
            
    new_aln.close()
    
fw = open("representative_annotations.tsv", 'w')
fw.write("taxon\trepresentative\n")

with open("representatives_major_lineage.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        fw.write(f"{l['taxon']}\t1\n")
        
fw.close()


## minor lins

In [20]:
minor_lin_sets = defaultdict(set)

for lin, seq_list in tqdm.tqdm(minor_lineages.items()):
    serotype = f"DENV{lin[0]}"
    if len(seq_list) > 5:
        new_seq_list = [x for x in seq_list if float(coverage[x]) >= 0.9]
        if len(new_seq_list) > 5:
            minor_lin_sets[lin] = get_sets(new_seq_list, serotype)
        else:
            minor_lin_sets[lin] = set(new_seq_list)
    else:
        minor_lin_sets[lin] = set(seq_list)
    
        
    tree = tree_dict[serotype]
    mrca = tree.mrca(taxon_labels=seq_list)
    found_sequence = False
    for i in mrca.child_nodes():
        if i.taxon:
            basal = i.taxon.label
            found_sequence = True
    
    if not found_sequence:
        dist_dict = {x:distances_from_root[serotype][x] for x in minor_lin_sets[lin]}
        basal = min(dist_dict, key=dist_dict.get)
    
    minor_lin_sets[lin].add(basal)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [06:11<00:00,  5.72s/it]


In [26]:
with open("representatives_minor_lineage.csv", 'w') as fw:
    
    fw.write('taxon,minor_lineage\n')
    
    for k,v in minor_lin_sets.items():
        for i in v:
            fw.write(f"{i},{k}\n")

In [42]:
minor_seq_names = defaultdict(list)
with open("representatives_minor_lineage.csv") as f:
    data = csv.DictReader(f)
    for l in data:
        sero = f"DENV{l['minor_lineage'][0]}"
        minor_seq_names[sero].append(l['taxon'])
        
for sero, names in minor_seq_names.items():
    new_aln = open(f"{sero}_minor_lineage.fasta",'w')
    for seq in SeqIO.parse(f"{sero}_final.fasta", "fasta"):
        if seq.id in names:
            SeqIO.write(seq, new_aln, "fasta")
            
    new_aln.close()